In [1]:
load("../Shared/orbits.sage") # Group orbits
load("../Shared/linalg.sage") # Auxiliary linear algebra
load("../Shared/cyclic_covers.sage") # Cyclic covers of function fields
load("../Shared/weil_poly_utils.sage") # Utility functions for Weil polynomials
from collections import defaultdict 

In [2]:
F = GF(2)
P.<x0,x1,x2> = PolynomialRing(F,order = 'lex')

In [3]:
S = [vector(t) for t in ProjectiveSpace(F, 2).rational_points()]
for v in S:
    v.set_immutable()

In [4]:
def apply_group_elem(g, x):
    x1 = g*x
    x1.set_immutable()
    return x1
def stabilizer(x):
    return GL(3, 2).subgroup([g.matrix().transpose() for g in vec_stab(Matrix(x)).gens()])
def optimized_rep(g):
    return g.matrix()

In [5]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep}

In [6]:
G = GL(3, F)
tree = build_orbit_tree(G, S,11, methods, verbose=False)

In [7]:
monos5 = [prod(x) for x in itertools.combinations_with_replacement(P.gens(), 5)]
coords5 = {x: vector(mu(*x) for mu in monos5) for x in S}

In [8]:
def vec_to_gen(v):
    return sum(v[i]*monos5[i] for i in range(len(monos5)))

In [9]:
# find curves with points in subsets (of size up to #P^2(F_2))) of P^2(F_2)
# they form an orbit under GL_3-action

curves = defaultdict(list)
perp = Matrix([coords5[x] for x in S])
for s in range(11):
    for vecs in green_nodes(tree, s):
        target = vector(F, (0 if x in vecs else 1 for x in S))
        for w in solve_right_iterator(perp, target):
            curves[(s,)].append(vec_to_gen(w))

In [10]:
[(s, len(curves[s])) for s in curves]

[((0,), 16384),
 ((1,), 16384),
 ((2,), 16384),
 ((3,), 32768),
 ((4,), 32768),
 ((5,), 16384),
 ((6,), 16384),
 ((7,), 16384)]

In [11]:
def count_by_ideal(gen, n):
    J = P.ideal([gen] + [y^(2^n) + y for y in P.gens()]) #relations from generator Q(x0,x1,x2) and F_2^n points
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [12]:
load("../Shared/pointcounts.sage")

In [13]:
for n in range(2, 5): #takes lot of time for > 4
    tmp = set(t[:n] for t in data)
    tmp2 = list(curves.keys())
    for s in tmp2:
        for gen in curves[s]:
            i = count_by_ideal(gen, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gen)
        del curves[s]
    print([(s, len(curves[s])) for s in curves])

[((0, 10), 168), ((0, 4), 5376), ((0, 2), 4984), ((0, 6), 2520), ((0, 8), 1120), ((0, 14), 8), ((0, 0), 2208), ((1, 9), 920), ((1, 1), 2184), ((1, 5), 5064), ((1, 3), 5120), ((1, 7), 2880), ((1, 13), 24), ((1, 11), 192), ((2, 10), 920), ((2, 2), 2184), ((2, 6), 5064), ((2, 4), 5120), ((2, 8), 2880), ((2, 14), 24), ((2, 12), 192), ((3, 5), 10104), ((3, 7), 10440), ((3, 9), 5400), ((3, 3), 4392), ((3, 11), 2040), ((3, 13), 360), ((3, 15), 24), ((3, 17), 8), ((4, 8), 10440), ((4, 10), 5400), ((4, 12), 2040), ((4, 6), 10104), ((4, 14), 360), ((4, 16), 24), ((4, 4), 4392), ((4, 18), 8), ((5, 9), 5064), ((5, 11), 2880), ((5, 13), 920), ((5, 7), 5120), ((5, 15), 192), ((5, 17), 24), ((5, 5), 2184), ((6, 12), 2880), ((6, 10), 5064), ((6, 8), 5120), ((6, 14), 920), ((6, 18), 24), ((6, 16), 192), ((6, 6), 2184), ((7, 13), 2520), ((7, 15), 1120), ((7, 9), 4984), ((7, 11), 5376), ((7, 7), 2208)]
[((0, 10, 18), 168), ((0, 4, 9), 1680), ((0, 4, 12), 1344), ((0, 4, 6), 2016), ((0, 4, 15), 336), ((0, 

In [14]:
lst = list(curves.keys())
for j in range(25):
    f = open('./data/' + str(j) + '.txt','w+')
    for key in lst[66*j:66*j + 66]:
        for gens in curves[key]:
            f.write(str(gens))
            f.write('\n')
    f.close()

In [15]:
len(curves.keys())

1650

In [16]:
1650/25

66